# Creating dataset for new categories: BUG and FEATURE

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/EECS_6448/Data


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/EECS_6448/Data


In [ ]:
import re
import pandas as pd

## Read the dataset for training of the new categories to be able to label these categories in the original URR dataset.

#### Assign column names for the categories dataset.

In [ ]:
df = pd.read_csv('Data/spam.csv', names=["App", "Source", "Author", "ReviewID", "Language", "Unnamed", "Star", "Date", "User", "Summary", "Review", "Category"])
df = df.drop("Unnamed", axis=1)
df['Review'] = df['Summary'] + ' ' + df['Review']
df = df.drop("Summary", axis=1)
df.head()

,App,Source,Author,ReviewID,Language,Star,Date,User,Review,Category
0,Ingress,Google Play,com.nianticproject.ingress,Z3A6QU9xcFRPRVlWTzVIT3lnekJJMDBlS3ZSQzJ3cVUyMU...,English,5,8/8/2016,Arnaud - Gustaf Valentine,NaN,s
1,Ingress,Google Play,com.nianticproject.ingress,Z3A6QU9xcFRPRnNoczVsTmIwUFVvbUNLYVlsbkkwZTlOck...,English,1,8/8/2016,Ezza Zaheer,NaN,s
2,Ingress,Google Play,com.nianticproject.ingress,Z3A6QU9xcFRPRWRFSWtDSkVBWkVvcFNjNDlzeEVFbzR3aH...,English,3,8/8/2016,Noraslinda Fauzi,NaN,u
3,Ingress,Google Play,com.nianticproject.ingress,Z3A6QU9xcFRPSG9qWHo5NjdUVzBGRUttOTBwdC1LNWtMcH...,English,5,8/7/2016,Blaise Tyrel Daga,iyot mani ot yawa mo bati ka ayo,s
4,Ingress,Google Play,com.nianticproject.ingress,Z3A6QU9xcFRPR1RVZHBIRTFJS0hNWnczeWR4SW1QUk9mWU...,English,4,8/7/2016,Thomas Svedell,explain permissions!! upd app to android 6.0 t...,bu


#### Assign a target value based on the category. Use 'f' for FEATURE and 'b' for BUG.

#### Similarly BUG dataset was created too.

In [ ]:
for row in df.itertuples():
    feat = df.at[row.Index, 'Category']
    if 'f' in feat:
        df.at[row.Index, 'Target'] = 1
    else:
        df.at[row.Index, 'Target'] = 0


In [ ]:
import unicodedata
import nltk
import string
from nltk.corpus import stopwords

nltk.download('stopwords')


STOPWORDS = stopwords.words('english')
TOK = nltk.tokenize.toktok.ToktokTokenizer()
PORTER_STEMMER = nltk.porter.PorterStemmer()
WHITELIST = string.digits + string.whitespace + string.ascii_letters


def remove_punctuation(text: str) -> str:
    # Replace accented chars with normal form
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore').strip()

    # Remove double space
    while '  ' in text:
        text = text.replace('  ', ' ')

    # Keep only A-Z, a-z, 0-9, space
    return ''.join(c for c in text if c in WHITELIST)


def remove_stopwords(text: str) -> str:
    tokens = [t.strip() for t in TOK.tokenize(text.lower())]
    return ' '.join(t for t in tokens if t not in STOPWORDS)


def stem(text: str) -> str:
    return ' '.join([PORTER_STEMMER.stem(word) for word in text.split()])


def preprocess(text: str) -> str:
    return stem(remove_stopwords(remove_punctuation(text)))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#### Preprocess the dataset.

In [ ]:
# Replace NaN values with 'nan'
df.fillna('nan', inplace=True)

df['Review_proc'] = df['Review'].apply(preprocess)

df.head()

,App,Source,Author,ReviewID,Language,Star,Date,User,Review,Category,Target,Review_proc
0,Ingress,Google Play,com.nianticproject.ingress,Z3A6QU9xcFRPRVlWTzVIT3lnekJJMDBlS3ZSQzJ3cVUyMU...,English,5,8/8/2016,Arnaud - Gustaf Valentine,nan,s,0.0,nan
1,Ingress,Google Play,com.nianticproject.ingress,Z3A6QU9xcFRPRnNoczVsTmIwUFVvbUNLYVlsbkkwZTlOck...,English,1,8/8/2016,Ezza Zaheer,nan,s,0.0,nan
2,Ingress,Google Play,com.nianticproject.ingress,Z3A6QU9xcFRPRWRFSWtDSkVBWkVvcFNjNDlzeEVFbzR3aH...,English,3,8/8/2016,Noraslinda Fauzi,nan,u,0.0,nan
3,Ingress,Google Play,com.nianticproject.ingress,Z3A6QU9xcFRPSG9qWHo5NjdUVzBGRUttOTBwdC1LNWtMcH...,English,5,8/7/2016,Blaise Tyrel Daga,iyot mani ot yawa mo bati ka ayo,s,0.0,iyot mani ot yawa mo bati ka ayo
4,Ingress,Google Play,com.nianticproject.ingress,Z3A6QU9xcFRPR1RVZHBIRTFJS0hNWnczeWR4SW1QUk9mWU...,English,4,8/7/2016,Thomas Svedell,explain permissions!! upd app to android 6.0 t...,bu,0.0,explain permiss upd app android 60 let user al...


#### Balance the dataset for training.

In [ ]:
# Count the occurrences of each class
class_counts = df['Target'].value_counts()

# Find the minimum count
min_count = class_counts.min()

# Sample the DataFrame for each class to balance the number of rows
df = pd.concat([df[df['Target'] == label].sample(min_count) for label in df['Target'].unique()])

# Shuffle the rows of the balanced DataFrame
df = df.sample(frac=1).reset_index(drop=True)

In [ ]:
class_counts

0.0    4205
1.0     295
Name: Target, dtype: int64

#### We use xgboost for training the model as it was suggested by the URR paper for categorization.

In [ ]:
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
import xgboost
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

#### Training the model for categorization on the 2 added categories.

In [ ]:
X = df['Review_proc']
y = df['Target']

# Split the dataset into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

tfidf_vec = TfidfVectorizer(ngram_range=(1, 3), max_features=5000)
tfidf_vec.fit(df['Review_proc'].to_numpy())

# TF-IDF transform
X_train = tfidf_vec.transform(X_train.to_numpy())
y_train = y_train.to_numpy()

X_test = tfidf_vec.transform(X_test.to_numpy())
y_test = y_test.to_numpy()

# Ensure the target variable is numeric
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.fit_transform(y_test)

classifier = xgboost.XGBClassifier(n_estimators=100)

# Train the classifier on the training set
classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = classifier.predict(X_test)

# Evaluate the accuracy of the classifier
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy on the test set: {accuracy:.4f}")

Accuracy on the test set: 0.7119


#### Loading the original URR dataset for categorization.

#### Here it is done for FEATURE but similarly we do it for BUG and add it in the dataset.

In [ ]:
dataset = pd.read_csv('Data/reviews.csv')
dataset['reviewTextProc'] = dataset['reviewText'].apply(preprocess)

In [ ]:
tfidf_vec = TfidfVectorizer(ngram_range=(1, 3), max_features=5000)
tfidf_vec.fit(dataset['reviewTextProc'].to_numpy())

X_new = tfidf_vec.transform(dataset['reviewTextProc'].to_numpy())

dataset['IS_FEATURE'] = classifier.predict(X_new)

dataset.drop('reviewTextProc', axis=1, inplace=True)

dataset.to_csv('Data/reviews.csv', index=False)

In [ ]:
dataset.head()

,_id,app,appVersion,reviewText,author,date,ratingStars,ArDoc classification,classification,sub-classification,...,IS_BATTERY,IS_PRICE,IS_APP USABILITY,IS_ANDROID VERSION,IS_UI,IS_LICENSING,IS_MEMORY,IS_SECURITY,IS_BUG,IS_FEATURE
0,57291b7c507c69054da8ebeb,2048,2.06,Keeps crashing Since I upgraded my android thi...,Astrid Elkesdottir,26/02/16,1.0,INFORMATION_GIVING,NaN,NaN,...,0,0,0,1,0,0,0,0,0,0
1,57291b7c507c69054da8ebea,2048,2.06,Addictive I first became a fan of this game wh...,Holly Youngstrom,22/04/16,5.0,INFORMATION_SEEKING,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
2,57291b7c507c69054da8ebe9,2048,2.06,Ad free port of the original open source game.,Mike McLean,01/05/16,5.0,OTHER,NaN,NaN,...,0,1,0,0,0,1,0,0,1,0
3,57291b7c507c69054da8ebec,2048,2.06,love it.. good excercise before i go to work..,ahmir Cruz,20/02/16,5.0,OTHER,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
4,57291b7f507c69054da8ebed,2048,2.06,"Slow Since latest update, it's moving very slo...",Zachary Burris,15/02/16,4.0,OTHER,NaN,NaN,...,0,0,0,0,1,0,0,0,1,0
